# MediaPipe Data Extraction

By providing the specific video in the "Set up system" section, this script will produce 

# Install MediaPipe

In [ ]:
pip install mediapipe opencv-python

# Set up system

Modify the "cap" variable to clarify the exact video that you're extracting information from.

In [ ]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

cap = cv2.VideoCapture("bench_press.mp4")


# Compute keypoints from the frame

In [ ]:
frame_idx = 0
keypoints = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Extract 3D coordinates for wrists, elbows, shoulders
        landmarks = results.pose_landmarks.landmark
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]

        keypoints.append({
            "frame": frame_idx,
            "lwrist": (left_wrist.x, left_wrist.y, left_wrist.z),
            "rwrist": (right_wrist.x, right_wrist.y, right_wrist.z),
            "lelbow": (left_elbow.x, left_elbow.y, left_elbow.z),
            "relbow": (right_elbow.x, right_elbow.y, right_elbow.z),
        })

    frame_idx += 1

cap.release()


In [ ]:
import pandas as pd

df = pd.DataFrame(keypoints)
df.to_csv("mediapipe_lift_trajectory.csv", index=False)


# Extract the bar information from the other keypoints

In [ ]:
df["bar_x"] = (df["lwrist"].apply(lambda p: p[0]) + df["rwrist"].apply(lambda p: p[0])) / 2
df["bar_y"] = (df["lwrist"].apply(lambda p: p[1]) + df["rwrist"].apply(lambda p: p[1])) / 2
df["bar_z"] = (df["lwrist"].apply(lambda p: p[2]) + df["rwrist"].apply(lambda p: p[2])) / 2


# Save the bar information

In [ ]:
import numpy as np
bar_data = np.loadtxt("mediapipe_lift_trajectory.csv", delimiter=",", skiprows=1)
bar_z_positions = bar_data[:, -1]  # assuming last column = bar_z
